## Walidacja wejścia i wyjścia z wykorzystaniem Pydantic

In [2]:
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import dotenv

dotenv.load_dotenv()

# 3a) Definicja schematu wyjścia (Pydantic)
class CityGuide(BaseModel):
    city: str = Field(..., description="Miasto, którego dotyczy przewodnik")
    summary: str = Field(..., description="Krótki opis miasta (2–3 zdania)")
    must_do: List[str] = Field(..., description="Lista 3–5 rzeczy do zrobienia")

# 3b) LLM + structured output
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(CityGuide)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś ekspertem od podróży. Odpowiadaj po polsku, zwięźle."),
    ("user", "Stwórz krótki przewodnik po {city} dla {days}-dniowej wizyty.")
])

chain = prompt | structured_llm

# 3c) (opcjonalnie) walidacja wejścia Pydantic + wywołanie łańcucha
from pydantic import BaseModel, Field

class GuideRequest(BaseModel):
    city: str = Field(min_length=2)
    days: int = Field(ge=1, le=7)

req = GuideRequest(city="Poznań", days=2)

result: CityGuide = chain.invoke(req.model_dump())  # <- dostajesz OBIEKT Pydantic!
print(result)                         # CityGuide(city=..., summary=..., must_do=[...])
print(result.model_dump_json(indent=2))  # JSON gotowy do zapisu/transportu

city='Poznań' summary='Poznań to jedno z najstarszych miast w Polsce, znane z bogatej historii, pięknej architektury i tętniącej życiem atmosfery. To idealne miejsce na krótki wypad, oferujące zarówno zabytki, jak i nowoczesne atrakcje.' must_do=['Odwiedź Stary Rynek i zobacz ratusz z koziołkami.', 'Spaceruj po Ostrówie Tumskim, historycznej wyspie z katedrą.', 'Zrelaksuj się w Parku Cytadela, gdzie znajdziesz muzea i piękne tereny zielone.', 'Spróbuj lokalnych specjałów w jednej z restauracji, np. pyry z gzikiem.', 'Zobacz Muzeum Narodowe z bogatą kolekcją sztuki.']
{
  "city": "Poznań",
  "summary": "Poznań to jedno z najstarszych miast w Polsce, znane z bogatej historii, pięknej architektury i tętniącej życiem atmosfery. To idealne miejsce na krótki wypad, oferujące zarówno zabytki, jak i nowoczesne atrakcje.",
  "must_do": [
    "Odwiedź Stary Rynek i zobacz ratusz z koziołkami.",
    "Spaceruj po Ostrówie Tumskim, historycznej wyspie z katedrą.",
    "Zrelaksuj się w Parku Cytadel